In [1]:
import pandas as pd
import ast

reports = pd.read_csv("data/sustainability_reports.csv")
scores = pd.read_csv("data/esg_scores.csv")
reports = reports.drop(columns=['Unnamed: 0'])

reports.head()

,filename,ticker,year,preprocessed_content,ner_entities,e_score,s_score,g_score,total_score
0,ASX_BSX_2020.pdf,BSX,2020,style guide colour colour use imagecolour prof...,"['bk%', 'rgb', 'un', 'el ectric mine consortiu...",3.16,18.00,11.83,32.98
1,ASX_BSX_2022.pdf,BSX,2022,sustainability report look mining green office...,"['murray street', 'west perth', 'west perth', ...",2.83,12.86,10.32,26.02
2,ASX_EXR_2022.pdf,EXR,2022,report environment social governance esg basel...,"['september', 'mongolia', 'australia', 'austra...",3.81,4.28,5.86,13.94
3,LSE_ADM_2019.pdf,ADM,2019,corporate social responsibilty report introduc...,"['david stevens', 'csr board', 'just over yea...",16.38,14.20,5.90,36.36
4,LSE_ADM_2020.pdf,ADM,2020,sustainability admiral commit maintain respons...,"['year', 'health & wellbeing', 'a -month', 'on...",15.89,13.51,5.38,34.78


In [3]:
reports['ticker'] = reports['ticker'].str.lower()
companies = reports['ticker'].unique()

for company in companies:
    rep = reports[reports['ticker'] == company]

    print("="*80)
    print(f"COMPANY: {company.upper()}")
    print("-"*80)

    # YEARS
    report_years = rep['year'].unique()

    print(f"Report years: {sorted(report_years)}")

    # NER ENTITY LIST LENGTHS
    if "ner_entities" in rep.columns:
        try:
            ner_lengths = rep['ner_entities'].apply(
                lambda x: len(ast.literal_eval(x)) if isinstance(x, str) else 0
            )
        except Exception as e:
            print("NER parse error:", e)
            ner_lengths = pd.Series([0]*len(rep))

        print(f"NER entity list lengths (n={len(ner_lengths)}):")
        print(ner_lengths.describe())
        print()
    else:
        print("No ner_entities column found.\n")

COMPANY: BSX
--------------------------------------------------------------------------------
Report years: [np.int64(2020), np.int64(2022)]
NER entity list lengths (n=2):
count      2.000000
mean     438.500000
std       34.648232
min      414.000000
25%      426.250000
50%      438.500000
75%      450.750000
max      463.000000
Name: ner_entities, dtype: float64

COMPANY: EXR
--------------------------------------------------------------------------------
Report years: [np.int64(2022)]
NER entity list lengths (n=1):
count     1.0
mean     93.0
std       NaN
min      93.0
25%      93.0
50%      93.0
75%      93.0
max      93.0
Name: ner_entities, dtype: float64

COMPANY: ADM
--------------------------------------------------------------------------------
Report years: [np.int64(2019), np.int64(2020), np.int64(2021)]
NER entity list lengths (n=3):
count      3.000000
mean     230.333333
std       37.072002
min      206.000000
25%      209.000000
50%      212.000000
75%      242.500000


In [16]:
import textwrap

def split_and_quote(text, width=500):
    """
    Split text into 500-character chunks.
    Each line:
      - starts with a double quote
      - ends with a double quote
      - contains no internal newlines
    """
    if pd.isna(text):
        return []
    
    # Remove literal newline characters
    text = (
        text.replace("\n", " ")    # actual newline
            .replace("\r", " ")    # carriage returns
    )

    # Wrap into fixed-width chunks
    chunks = textwrap.wrap(
        text,
        width=width,
        break_long_words=True,
        drop_whitespace=True
    )

    # Wrap each chunk in quotes
    quoted_chunks = [f"\"{chunk}\"" for chunk in chunks]
    return quoted_chunks


# Apply to each row
reports["processed_lines"] = reports['preprocessed_content'].apply(split_and_quote)

# Join lines with real newlines (not the literal string "\n")
reports["processed_text_block"] = reports["processed_lines"].apply(
    lambda lines: "\n".join(lines)
)

# Show example
print(reports['processed_text_block'].iloc[0])

"style guide colour colour use imagecolour professional print pantone gold black single colour use blackreverse colour use black dark photo use reverse logo dark photo read photo gold foil cmyk crop photoof gold foil use normal logo light photo read sustainability content introduction manage director report highlight report story khoa sustainability strategy journey share blackstone vision responsible sustainable mining understand sustainability mean blackstone stakeholder share value material"
"issue responsibility sustainability sustainable development goal future action activity health safety wellbee review develop safety understanding skill pandemic response dapte technology reduce frontline risk pandemic esponsible business good governance troduction board vernment engagement permit environment climate change create positive legacy community environment limate change een nickel integrate value chain renewable power pressure oxidation pox ossil fuel consortium calendar year emissio

In [17]:
processed_text = reports['processed_text_block']
processed_text

0      "style guide colour colour use imagecolour pro...
1      "sustainability report look mining green offic...
2      "report environment social governance esg base...
3      "corporate social responsibilty report introdu...
4      "sustainability admiral commit maintain respon...
                             ...                        
861    "absolute software corporate social responsibi...
862    "absolute software corporate social responsibi...
863    "esg performance summary environmental social ...
864    "corporate social responsibility report mornea...
865    "corporate social responsibility report improv...
Name: processed_text_block, Length: 866, dtype: object